In [ ]:
%matplotlib inline

import numpy as np
import pickle
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from quick_pp.objects import Project

# Load well from saved file
project_name = "VOLVE"
project_path = rf"data\04_project\{project_name}.qppp"
project = Project().load(project_path)
project.get_well_names()

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

# Load FZI ROCK_FLAG
with open(rf'data\04_project\{project_name}\outputs\fzi_rt_model.qppm', 'rb') as file:
    fzi_rt_model = pickle.load(file)

# Load FZI model
with open(rf'data\04_project\{project_name}\outputs\fzi_model.qppm', 'rb') as file:
    fzi_model = pickle.load(file)

# Load FZI cutoffs
with open(rf'data\04_project\{project_name}\outputs\fzi_cutoffs.json', 'rb') as file:
    fzi_cutoffs = json.load(file)

# Quick PP Interpretation

In [ ]:
from quick_pp.lithology.sand_shale import SandShale
from quick_pp.porosity import *
from quick_pp.qaqc import *
from quick_pp.saturation import *
from quick_pp.plotter.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import *
from quick_pp.plotter.plotter import plotly_log
from quick_pp.utils import *

In [ ]:
for well_name in project.get_well_names():
    # Load well object
    well = project.get_well(well_name)
    well_data = well.data
    well_data.columns = well_data.columns.str.replace('LFP_', '')

    # Clean up data
    well_data = well_data.dropna(subset=['NPHI', 'RHOB', 'RT', 'GR'])
    well_data = badhole_flagging(well_data)
    well_data = mask_outside_threshold(well_data, fill=True)

    # Initialize lithology model
    args = {
    'litho_model': 'ss',
    # 'dry_clay_point': (.3, 2.7),
    'wet_clay_point': (0.45, 2.45),
    'sw_water_salinity': 15000,
    'sw_m': 1.85,
    'sw_n': 1.85,
    'hc_corr_angle': neu_den_xplot_hc_correction_angle(rho_water=1.0, rho_hc=0.8, HI_hc=0.9),
    'hc_buffer': 0.01,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
        )
    }
    ss_model = SandShale(**args)
    vsand, vcld = ss_model.estimate_lithology(
        well_data['NPHI'], well_data['RHOB']
    )
    args.update(ss_model.__dict__)
    well.update_config(args)  # Save lithology model to well

    # Implement hydrocarbon correction
    vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=140)
    nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
        well_data['NPHI'], well_data['RHOB'], vsh_gr=vsh_gr,
        dry_min1_point=args['dry_sand_point'],
        dry_clay_point=args['dry_clay_point'],
        corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
    )
    
    # Correct density log
    rhob_corr = den_correction(nphihc, well_data['GR'], vsh_gr=vsh_gr, alpha=0.1)
    badhole_flag =  np.where(abs(well_data['RHOB'] - rhob_corr) > 0.2, 1, 0)
    rhob_corr = np.where((badhole_flag == 1) & (hc_flag == 0), rhob_corr, rhobhc)

    # Estimate lithology
    ss_model = SandShale(**args)
    vsand, vcld = ss_model.estimate_lithology(
        nphihc, rhob_corr
    )

    # Estimate porosity
    phit = neu_den_xplot_poro(
        nphihc, rhob_corr, model='ss',
        dry_min1_point=args['dry_sand_point'],
        dry_clay_point=args['dry_clay_point'],
    )

    rho_ma = rho_matrix(vsand, vcld)
    phid = density_porosity(rhob_corr, rho_ma)
    
    # Normalize lithology
    volumes = dict(vcld=vcld, vsand=vsand)
    volumes = normalize_volumetric(phit, **volumes)
    vcld, vsand = volumes['vcld'], volumes['vsand']

    # Calculate vclb: volume of clay bound water and phie
    clay_phit = clay_porosity(rhob_corr, args['dry_clay_point'][1])
    vclb = vcld * clay_phit
    vclay = vcld + vclb
    phie = phit - vclb

    # Predict rock type
    well_data['LOG_RT'] = np.log10(well_data['RT'])
    well_data['NDI'] = (2.95 - well_data['RHOB']) / 1.95
    well_data['GRN'] = MinMaxScaler().fit_transform(well_data[['GR']])
    input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN']
    rock_flag_ml = fzi_rt_model.predict(well_data[input_features])
    
    # Predict permeability
    temp_df = well_data.copy()
    temp_df['ROCK_PRED'] = rock_flag_ml
    input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN', 'ROCK_PRED']
    temp_df[input_features] = imp_mean.fit_transform(temp_df[input_features])
    fzi_ml = 10**(fzi_model.predict(temp_df[input_features]))
    perm_ml = calc_fzi_perm(fzi_ml, phit)
    perm_ml = np.where(perm_ml < 0, 0, perm_ml)

    # Update rock type
    rock_flag_ml = rock_typing(fzi_ml, cut_offs=fzi_cutoffs)

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')  # Arbitrarily use MD instead of TVD
    water_salinity = args['sw_water_salinity']
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    b = estimate_b_waxman_smits(temp_grad, rw)
    qv = estimate_qv(vcld, phit, cec_clay=.1)
    m = args['sw_m']

    swt = waxman_smits_saturation(well_data['RT'], rw, phit, qv).clip(0, 1)

    # Update ZONES
    well_data = zone_flagging(well_data)
    zones = well_data.ZONES

    # Estimate reservoir summary
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclay, phit, swt, perm_ml, zones, cutoffs=args['ressum_cutoffs'])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhob_corr
    well_data['HC_FLAG'] = hc_flag
    well_data['VSAND'] = vsand
    well_data['VCLAY'] = vclay
    well_data['PHIT'] = phit
    well_data['PHIE'] = phie
    well_data['PHID'] = phid
    well_data['RW'] = rw
    well_data['B'] = b
    well_data['QV'] = qv
    well_data['M'] = m
    well_data['SWT'] = swt
    well_data['BVW'] = swt * phie
    well_data['PERM'] = perm_ml
    well_data['ROCK_FLAG'] = rock_flag_ml
    well_data['VHC'] = (phit * (1 - swt)).clip(0, 1)
    well_data['VSH_GR'] = vsh_gr

    # Save the results to the well
    well.update_data(well_data)
    well.update_ressum(ressum_df)
    project.save_well(well)


In [ ]:
# Plot individual results
final_df = project.get_all_data()
output_folder = r'data\04_project\outputs'
for well, data in final_df.groupby('WELL_NAME'):
    fig = plotly_log(data, well_name=well, column_widths=[1, 1, 1, 1, 1, 1, .3, 1, 1])
    # fig.show(config=dict(scrollZoom=True))
    fig.write_html(rf"{output_folder}\{well}_log.html", config=dict(scrollZoom=True))

In [ ]:
from quick_pp.qaqc import quick_compare

compare_df, fig = quick_compare(final_df, return_fig=True)